In [ ]:
import cv2
import ipywidgets as widgets
import traitlets
from IPython.display import display
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
import jetson_inference
import jetson_utils
import threading
import time

# Initialize the object detection network
net = jetson_inference.detectNet("ssd-mobilenet-v2", threshold=0.5)

# Initialize the camera
camera = CSICamera(width=224, height=224)
camera.running = True

# Create image preview widgets
camera_widget = widgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

def update_image():
    while True:
        # Capture and process the image from the camera
        img = camera.value.copy()
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # Convert to BGR format

        # Convert image to CUDA format for detection
        img_cuda = jetson_utils.cudaFromNumpy(img)
        
        # Perform object detection
        detections = net.Detect(img_cuda)
        img_cuda = jetson_utils.cudaToNumpy(img_cuda)
        img = cv2.cvtColor(img_cuda, cv2.COLOR_BGR2RGB)  # Convert back to RGB format

        # Draw detection results
        for detection in detections:
            x1, y1, x2, y2 = int(detection.Left), int(detection.Top), int(detection.Right), int(detection.Bottom)
            label = net.GetClassDesc(detection.ClassID)
            color = (0, 255, 0)  # Green color for bounding boxes
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Update the image widget
        camera_widget.value = bgr8_to_jpeg(img)

        # Sleep for a short period to control the update rate
        time.sleep(0.5)  # 100 milliseconds

# Start the update thread
update_thread = threading.Thread(target=update_image, daemon=True)
update_thread.start()

# Display the widgets
display(camera_widget)
